In [1]:
!pip3 install tabula-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 11.4 MB/s 


In [2]:
path = "/content/drive/Othercomputers/My MacBook Pro/data-science/projects/pools-dashboard/"


In [3]:
import pandas as pd
import tabula
import re
import numpy as np

In [33]:
def process(df):
    # rename
    # print('hi', df.columns.values)
    df = df.copy().reset_index()
    df_cols = list(df.columns)
    a = [re.sub('\\r|\&|\/|\.', '', x).strip() for x in df_cols]
    a = ["_".join(x.split()).lower() for x in a]
    df.columns = a

    # datatypes
    df['amount'] = df['amount'].str.split(' ').str[-1]
    df['payout_winnings'] = df['payout_winnings'].str.split(' ').str[-1]

    df['transaction_date_time'] = df['transaction_date_time'].str.split('\\r').str[0] + ' ' + \
                                  df['transaction_date_time'].str.split('\\r').str[1]
    df['draw_eventdate_time'] = df['draw_eventdate_time'].str.split('\\r').str[0] + ' ' + \
                                df['draw_eventdate_time'].str.split('\\r').str[1]
    df['status_receiptno'] = df['status_receiptno'].str.split('\\r').str[0]
    df['type'] = df['type'].str.split('\\r').str[0]

    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
    df['payout_winnings'] = pd.to_numeric(df['payout_winnings'], errors='coerce')
    df['transaction_date_time'] = pd.to_datetime(df['transaction_date_time'], format='%d %b %Y %I:%M %p')
    df['draw_eventdate_time'] = pd.to_datetime(df['draw_eventdate_time'], format='%d %b %Y %I:%M %p')

    def date_filter(df):
        if (df['type'] == 'Withdrawal') or (df['type'] == 'Deposit'):
            return df['transaction_date_time']
        else:
            return df['draw_eventdate_time']

    df['datetime'] = df.apply(date_filter, axis=1)

    df = df.drop(['channel'], axis=1)

    return df


def football_table(df):
    df = df.copy()
    df = df.loc[(df['type'] == 'Football') | (df['type'] == 'Deposit') | (df['type'] == 'Withdrawal')]
    df = df.sort_values(by=['datetime'])

    # df['returns']= df[['payout_winnings','amount']].max(axis=1)
    # df['returns']= np.where(df['payout_winnings']== 0,  df['returns']*-1, df['returns'])
    # df['returns'] = np.where(df['type'] == 'Withdrawal', df['returns'] * -1, df['returns'])

    def returns_filter(df):
        if (df['type'] == 'Football') and (df['status_receiptno'] == 'Settled'):
            return df['payout_winnings'] - df['amount']
        elif df['type'] == 'Withdrawal':
            return -df['amount']
        elif (df['type'] == 'Deposit'):
            return df['amount']

    df['returns'] = df.apply(returns_filter, axis=1)
    df['cum_sum'] = df['returns'].cumsum()
    df['perc_returns'] = df['returns'] / df['cum_sum'].shift(periods=1)

    # selection_details split into columns
    df['selection_details'] = df['selection_details'] \
        .apply(lambda x: re.sub('\\r|\&|\@|\([a-zA-Z]+\)', ' ', x).strip())
    df['live'] = np.where(df['selection_details'].str.contains('live'), 1, 0)

    df['league'] = df['selection_details'].str.split('-').str[0]
    df['match'] = df['selection_details'].str.split('-').str[1]
    df['home'] = df['match'].str.split('vs').str[0]
    df['away'] = df['match'].str.split('vs').str[1]
    df['bet_side'] = df['selection_details'].str.split('-').str[2]
    df[['league', 'match', 'bet_side', 'home', 'away']] = \
        df[['league', 'match', 'bet_side', 'home', 'away']].apply(lambda x: x.str.strip())
    df['sub_type'] = df['bet_side'].str.split(' ').str[0]
    df['bet_side'] = df['bet_side'].str.split(' ').str[1]
    df['odds'] = df['selection_details'].str.split(' ').str[-1]
    df['odds'] = pd.to_numeric(df['odds'], errors='coerce')
    df['imp_prob'] = 1 / df['odds']

    df['home'].fillna('Unknown', inplace=True)
    df['away'].fillna('Unknown', inplace=True)

    def bet_filter(df):
        if df['bet_side'] == 'Draw':
            return 'D'
        elif str(df.bet_side) in str(df.home):
            return 'H'
        else:
            return 'A'

    df['bet_side'] = df.apply(bet_filter, axis=1)

    df = df.drop(['selection_details'], axis=1)

    return df


def win_ratio_table(df):
    df = df.copy()
    df = df.loc[df['status_receiptno'] == 'Settled']
    df['win_ind'] = np.where(df['returns'] > 0, 1, 0)
    return df

def return_by_match_table(df):
    df = df.copy()
    df = df[['datetime', 'home', 'away', 'amount', 'payout_winnings', 'imp_prob', 'bet_side', 'win_ind', 'returns']]
    df['match_total_bet'] = df.groupby(['home', 'away', 'datetime'])['amount'].transform(np.sum)
    df['match_total_returns'] = df.groupby(['home', 'away', 'datetime'])['payout_winnings'].transform(np.sum)
    df['bet_per_match'] = df['amount'] / df['match_total_bet']
    df['imp_prob_weighted'] = np.round(df['imp_prob'] * df['bet_per_match'],2)
    df['date'] = df['datetime'].dt.date

    # groupby
    # aggregate bets for each match, since same event
    bet_side_one_hot = pd.get_dummies(df['bet_side'])
    df = pd.concat([df, bet_side_one_hot], axis=1)
    # df[['bet_side_A', 'bet_side_D', 'bet_side_H']] = df[['bet_side_A', 'bet_side_D', 'bet_side_H']]\
    df[['A', 'D', 'H']] = df[['A', 'D', 'H']].apply(lambda x: x * df['bet_per_match'])

    df = df.drop(['match_total_returns', 'match_total_bet', 'bet_per_match', 'imp_prob'], axis=1)
    df['num_of_bets'] = 1
    df = df.groupby(['home', 'away', 'date']).sum()

    df['frac_bets_win'] = df['win_ind'] / df['num_of_bets']
    df['agg_win_ind'] = np.where((df['payout_winnings'] - df['amount']) > 0, 1, 0)
    a = df[['A', 'D', 'H']]
    df = df.assign(side_agg=a.idxmax(axis=1), max_side_bet=a.max(axis=1))
    df['exact_frac_win'] = df['max_side_bet'] * df['agg_win_ind']
    df = df.reset_index()
    df['match_name'] = df['home'] + ' v ' + df['away']
    return df


def return_by_match_table_simple(df):
    df = df[['match_name', 'date', 'returns']]
    return df


In [34]:
def fetch_eg_csv(path):
    # note that file path starts from pools-dashboard directory
    df_list = tabula.read_pdf(path + "data/TransactionHistory.pdf", pages='all')
    df = pd.concat(df_list)
    return df

def convert_to_df(df_input):
    if df_input is not None:
        df = df_input.copy()
        df = process(df)
    return df


def df_to_match(df):
    if df is not None:
        df = football_table(df)
        df_wr = win_ratio_table(df)
        # wr= charts.win_ratio(df_wr)
        df_match = return_by_match_table(df_wr)
    return df_match

In [6]:
df = fetch_eg_csv(path)

Nov 26, 2022 4:40:42 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Nov 26, 2022 4:40:43 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Nov 26, 2022 4:40:43 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Nov 26, 2022 4:40:43 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [15]:
df.head()

,TRANSACTI\rON DATE &\rTIME,TYPE,CHANNEL,SELECTION / DETAILS,AMOUNT,DRAW /\rEVENT\rDATE &\rTIME,STATUS / RECEIPT\rNO.,PAYOUT /\rWINNINGS
0,11 Aug 2022\r12:22 AM,Deposit,-,eNETS Transaction Fee,-$ 0.80,-,-,-
1,11 Aug 2022\r12:22 AM,Deposit,-,Deposit - eNETS,$ 110.80,-,-,-
2,11 Aug 2022\r02:56 AM,TOTO\rTOTO\rOrdinary,Web,"13,16,18,33,42,45\rSelf Pick",$ 1.00,11 Aug 2022\r06:30 PM,Settled\rL/0111562/0000001,$ 0.00
3,11 Aug 2022\r03:01 AM,Football,Web,English Premier\r(Outrights/Specials) - Englis...,$ 1.00,29 May 2023\r12:00 AM,Placed\rO/0115148/0000089,-
4,11 Aug 2022\r03:01 AM,Football,Web,English Premier\r(Outrights/Specials) - Englis...,$ 6.00,29 May 2023\r12:00 AM,Placed\rO/0115148/0000088,-


In [35]:
df_clean = process(df)
df_match = df_to_match(df_clean)
df_match.head()

,home,away,date,amount,payout_winnings,win_ind,returns,imp_prob_weighted,A,D,H,num_of_bets,frac_bets_win,agg_win_ind,side_agg,max_side_bet,exact_frac_win,match_name
0,Argentina,Saudi Arabia,2022-11-22,7.0,0.00,0,-7.00,0.90,0.0,0.0,1.0,2,0.0,0,H,1.0,0.0,Argentina v Saudi Arabia
1,Arsenal,Fulham,2022-08-28,3.0,5.41,1,2.41,0.70,0.0,0.0,1.0,1,1.0,1,H,1.0,1.0,Arsenal v Fulham
2,Arsenal,Liverpool,2022-10-09,2.0,0.00,0,-2.00,0.37,0.5,0.5,0.0,2,0.0,0,A,0.5,0.0,Arsenal v Liverpool
3,Arsenal,Tottenham,2022-10-01,1.0,0.00,0,-1.00,0.31,1.0,0.0,0.0,1,0.0,0,A,1.0,0.0,Arsenal v Tottenham
4,Atletico Madrid,Porto,2022-09-08,1.0,1.75,1,0.75,0.57,0.0,0.0,1.0,1,1.0,1,H,1.0,1.0,Atletico Madrid v Porto


In [36]:
df_match_plot = df_match[['match_name', 'date', 'returns','imp_prob_weighted']]

In [20]:
import plotly.express as px
def scatter_returns(df, x, y):
    fig = px.scatter(df, x=x, y=y, hover_data=['match_name'],
                     title='Match returns over match day ')
    return fig

In [62]:
 
fig = px.scatter(df_match_plot, x=['date'], y='returns', hover_data=['match_name'], color='imp_prob_weighted',
                    title='Match returns over match day ', marginal_y='violin')
fig.show()

In [56]:
# archie
def archie_score():
  df_archie= df_match[['match_name','imp_prob_weighted', 'win_ind']]
  num_bets = df_archie.shape[0]
  exp_win = df_archie['imp_prob_weighted'].sum()
  num_win_bets = df_archie[df_archie['win_ind'] ==1]['win_ind'].count()
  archie_score = num_bets*(num_win_bets - exp_win)**2/(exp_win * (num_bets - exp_win))

  if archie_score <= 0.3:
    chance = 0.58
  elif archie_score <= 0.5:
    chance = 0.48
  elif archie_score <= 1:
    chance = 0.32
  elif archie_score <= 1.5:
    chance = 0.22
  elif archie_score <= 2:
    chance = 0.16
  elif archie_score <= 2.5:
    chance = 0.11
  elif archie_score <= 3:
    chance = 0.08
  elif archie_score <= 4:
    chance = 0.05
  elif archie_score <= 5:
    chance = 0.03
  elif archie_score <= 8:
    chance = 0.01
  else:
    chance = 0

  return archie_score, chance

In [9]:
df_clean[df_clean['status_receiptno']=='Placed'].head()['selection_details']

3    English Premier\r(Outrights/Specials) - Englis...
4    English Premier\r(Outrights/Specials) - Englis...
5    English Premier\r(Outrights/Specials) - Englis...
6    Spanish League\r(Outrights/Specials) - Spanish...
7    Italian League (Outrights/Specials) -\rItalian...
Name: selection_details, dtype: object

In [10]:
def open_bets(df):
  df = df.copy()
  df = df.loc[(df['type'] == 'Football') | (df['type'] == 'Deposit') | (df['type'] == 'Withdrawal')]
  df = df.sort_values(by=['datetime'])
  df= df[df['status_receiptno']=='Placed']
  df = df[['draw_eventdate_time', 'selection_details','amount']]
  # make odds and win column
  df['odds'] = df[['selection_details']].apply(lambda x: x.str.split('@').str[-1])
  df['odds'] = pd.to_numeric(df['odds'])
  df['potential_win'] = df['odds'] * df['amount']
  # match column
  df['match'] = df['selection_details'] \
      .apply(lambda x: re.sub('\\r|\&|\@|\([a-zA-Z]+\)', ' ', x).strip()) 
  df['match'] = df['match'].str.split('-').str[1] + df['match'].str.split('-').str[2] 
  df['match'] =  df['match'].str.rsplit(' ',1).str[0]
  df= df.drop(columns=['selection_details'])
  # add totals
  df= df[['draw_eventdate_time','match','odds','amount','potential_win']]
  df['draw_eventdate_time'] = pd.to_datetime(df['draw_eventdate_time']).dt.normalize()
  df=df.reset_index(drop=True)
  return df

In [11]:
df_open = open_bets(df_clean)
df_open.head()

,draw_eventdate_time,match,odds,amount,potential_win
0,2022-11-27,France vs Denmark 1X2 France,1.63,2.0,3.26
1,2022-11-27,Argentina vs Mexico 1X2 Argentina,1.48,3.0,4.44
2,2022-11-27,Japan vs Costa Rica 1X2 Japan,1.35,2.0,2.70
3,2022-11-29,W Cup: Group A Winner Group Winner Holland,1.42,1.0,1.42
4,2022-11-29,W Cup: Group A Qualifier Group Qualifier Hol...,1.14,5.0,5.70


In [ ]:
df_wr = win_ratio_table(df_clean)
df_wr.head()